In [1]:
import pandas as pd
import shorthand as shnd

s = shnd.Shorthand(
    entry_syntax="shorthand/resources/default_entry_syntax.csv",
    link_syntax="shorthand/resources/default_link_syntax.csv",
    syntax_case_sensitive=False
)
'''
parsed = s.parse_text(
    'shorthand/test_data/manual_annotation.shnd',
    skiprows=2,
    comment_char='#'
)
'''

parsed = s.parse_text(
    'shorthand/test_data/single_column.shnd',
    item_separator='__',
    default_entry_prefix='wrk',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    skiprows=0,
    comment_char='#',
    drop_na=False
)

links = parsed.links
links.iloc[60:]
'''
print('       strings', parsed.strings.memory_usage(deep=True).sum()/1000, 'kb')
print('         links', parsed.links.memory_usage(deep=True).sum()/1000, 'kb')
print('resolved links', parsed.resolve_links().memory_usage(deep=True).sum()/1000, 'kb')
'''
entry_type = parsed.id_lookup('link_types', 'entry')
entry_string_ids = parsed.links.loc[parsed.links['link_type_id'] == entry_type, 'tgt_string_id']

parsed.resolve_links().query('src_string.str.contains("nasa")').query('src_node_type != "shorthand_text"').query('tgt_node_type != "shorthand_text"')

parsed.resolve_links().query('link_type == "cited"').merge(parsed.links, left_index=True, right_index=True)

s = parsed.synthesize_shorthand_entries('wrk', fill_spaces=True)

check = pd.Series([
    'asmith_bwu__1999__s_bams__101__803__xxx',
    'asmith_bwu__1998__s_bams__100__42__yyy',
    'bjones__1975__s_jats__90__1__!',
    'bwu__1989__t_long|title__!__80__!',
    'Some|Author__1989__t_A|Title|With|\\#__!__!__!',
    'asmith_bwu__2008__s_bams__110__1__zzz'
])

(check == s).all()

True

In [4]:
import pandas as pd
import shorthand as shnd
from bibtexparser.bparser import BibTexParser as _BibTexParser

bibtex_parser = _BibTexParser(common_strings=True)
with open("shorthand/test_data/bibtex_test_data_short.bib", encoding='utf8') as f:
    bibdatabase = bibtex_parser.parse_file(f)

data = pd.DataFrame(bibdatabase.entries)

s = shnd.Shorthand(
    entry_syntax="shorthand/resources/default_bibtex_syntax.csv",
    syntax_case_sensitive=False
)

parsed = s.parse_items(
    data.iloc[:4],
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)


0                                                   638
1                                                   641
2                                                   311
3                                                   323
4                                                   261
5                                                   280
6                                                   451
7                                                   453
8                                                  1962
9                                    Newkirk, Gordon A.
10                                        Eddy, John A.
11                                     Wiin-Nielsen, A.
12                                    Lally, Vincent E.
13                                               Nature
14                               Monthly Weather Review
15                                               Tellus
16      Bulletin of the American Meteorological Society
17                                              

In [7]:

    (check == synthesized).all()

True